In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Notes

* We are using dates to split the in-sample and out-of-sample for the walkforward, which  will introduce a bug where the preprocessing in each period reduces the number of trades.  
* TODO: refactor to address, by using bars instead of dates

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('maperiod:%2d, %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

## Analyzers

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.maperiod = self.strategy.params.maperiod
        self.equity = []
    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):

        if trade.isclosed:
            self.equity.append(self.strategy.broker.getvalue())
            self.ntrade += 1
            
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        equity = np.asarray([self.init_cash,] + self.equity)

        outp = {
            'params': (self.maperiod),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade,
            'trades': np.diff(equity).tolist(),
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

## In-sample Parameters

In [ ]:
periods = [
{'run': 1, 'oos': ('2017-04-01', '2017-05-01'), 'is': ('2017-01-01', '2017-04-01')},
{'run': 2, 'oos': ('2017-05-01', '2017-06-01'), 'is': ('2017-02-01', '2017-05-01')},
{'run': 3, 'oos': ('2017-06-01', '2017-07-01'), 'is': ('2017-03-01', '2017-06-01')},
{'run': 4, 'oos': ('2017-07-01', '2017-08-01'), 'is': ('2017-04-01', '2017-07-01')},
{'run': 5, 'oos': ('2017-08-01', '2017-09-01'), 'is': ('2017-05-01', '2017-08-01')},
{'run': 6, 'oos': ('2017-09-01', '2017-10-01'), 'is': ('2017-06-01', '2017-09-01')},
{'run': 7, 'oos': ('2017-10-01', '2017-11-01'), 'is': ('2017-07-01', '2017-10-01')},
{'run': 8, 'oos': ('2017-11-01', '2017-12-01'), 'is': ('2017-08-01', '2017-11-01')},
{'run': 9, 'oos': ('2017-12-01', '2018-01-01'), 'is': ('2017-09-01', '2017-12-01')},
{'run': 10, 'oos': ('2018-01-01', '2018-02-01'), 'is': ('2017-10-01', '2018-01-01')},
{'run': 11, 'oos': ('2018-02-01', '2018-03-01'), 'is': ('2017-11-01', '2018-02-01')},
{'run': 12, 'oos': ('2018-03-01', '2018-04-01'), 'is': ('2017-12-01', '2018-03-01')},
{'run': 13, 'oos': ('2018-04-01', '2018-05-01'), 'is': ('2018-01-01', '2018-04-01')},
{'run': 14, 'oos': ('2018-05-01', '2018-06-01'), 'is': ('2018-02-01', '2018-05-01')},

]

best_oos_params = {}
for oos_is in periods:
    start_date, end_date = oos_is['is']
    run_num = oos_is['run']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
        TestStrategy,
        maperiod=range(10, 31, 5))
    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.optreturn = False
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    best_param, best_profit, best_trades, best_equity = best_result_from_cerebro_opti_run(result)

    best_oos_params[run_num] = {'train_param': best_param, 
                                      'train_profit': best_profit, 
                                      'train_numtrades': best_trades, 
                                      'train_tradeslist': best_equity, 
                                      'train_period': oos_is['is'],
                                      'test_period': oos_is['oos'],
                                     }
    
    print('')

print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

In [ ]:
trades_list = np.asarray(best_oos_params[1]['train_tradeslist'])
cumsum = trades_list.cumsum()
plt.plot(cumsum)

## Out-of-sample Parameters

In [ ]:
for run_num, best in best_oos_params.items():
    start_date, end_date = best['test_period']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    cerebro.addstrategy(TestStrategy, maperiod=best['train_param'])

    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    r = result[0].analyzers.decisive.get_analysis()
    best_oos_params[run_num]['test_numtrades'] = r['ntrade']
    best_oos_params[run_num]['test_tradeslist'] = r['trades']
    best_oos_params[run_num]['test_profit'] = r['profit']
    

In [ ]:
print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

## Walkforward

In [ ]:
def days_from_date_tuple(mytuple):
    insample_days_start = datetime.datetime.strptime(mytuple[0], '%Y-%m-%d')
    insample_days_end = datetime.datetime.strptime(mytuple[1], '%Y-%m-%d')
    insample_days = (insample_days_end - insample_days_start).days
    return insample_days

oos_trades = []
wfe_list = []
for run_num, values in best_oos_params.items():
    oos_trades_list = values['test_tradeslist']
    oos_trades.extend(oos_trades_list)
    
    insample_days = days_from_date_tuple(values['train_period'])
    oos_days = days_from_date_tuple(values['test_period'])

    insample_annual_profit = 365/insample_days * values['train_profit']
    oos_annual_profit = 365/oos_days * values['test_profit']

    walkforward_efficiency_pct = round(oos_annual_profit * 100 / insample_annual_profit, 1)
    
    wfe_list.append(walkforward_efficiency_pct)

    print('Run {} WFE:{}'.format(run_num, walkforward_efficiency_pct))
    print('\t IS_DAYS: {} OOS_DAYS: {} IS_PROFIT: {:0.2f} OOS_PROFIT: {:0.2f}'.format(insample_days, oos_days,
                                                                            insample_annual_profit, oos_annual_profit))

print('Average WFE: {}%'.format(np.asarray(wfe_list).mean()))
starting_equity = 20000
plt.plot(np.asarray(oos_trades).cumsum() + starting_equity)
plt.title('Walkforward Equity Curve')